In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def reduce_mem_usage(df):
    #iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    #create a dataframe and optimize its memory usage
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
train_df = import_data('../input/tabular-playground-series-nov-2021/train.csv')
test_df = import_data('../input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
train = train_df.drop(['id'], axis=1)
test1 = test_df.drop(['id'], axis=1)

In [ ]:
train

In [ ]:
train.dtypes

In [ ]:
train.info()

In [ ]:
test1

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()

'''
best_features = ['f1', 'f10', 'f11', 'f14', 'f15', 'f16', 'f17', 'f2', 'f20', 'f21', 'f22', 'f24', 
                 'f25', 'f26', 'f27', 'f28', 'f3', 'f30', 'f31', 'f32', 'f33', 'f34', 'f36', 'f37', 
                 'f4', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f5', 
                 'f50', 'f51', 'f53', 'f54', 'f55', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f64', 
                 'f66', 'f67', 'f70', 'f71', 'f76', 'f77', 'f8', 'f80', 'f81', 'f82', 'f83', 'f87', 
                 'f89', 'f9', 'f90', 'f91', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98']
'''

train_copy = train.copy()

X = train_copy.drop(['target'], axis=1)
#X = np.float32(train_copy2)

#X = pd.DataFrame(scaler.fit_transform(train_copy2))
#X = np.float32(X)


y = train_copy.target

In [ ]:
X

In [ ]:
y

y.shape

In [ ]:
#test = test1.drop(['id'], axis=1)
test = test1.copy()

#X_test = scaler.transform(test)

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error, r2_score, roc_auc_score
from optuna.integration import XGBoostPruningCallback

def objective(trial):
    
    params = {
        "objective": trial.suggest_categorical('objective', ["binary:logistic"]),
        "eval_metric": trial.suggest_categorical('eval_metric', ["auc"]),
        "use_label_encoder": trial.suggest_categorical('use_label_encoder', [False]),
        "n_estimators": trial.suggest_categorical('n_estimators', [40000]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 0.15, 1.0),
        "subsample": trial.suggest_float('subsample', 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float('colsample_bytree', 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 8),
        "booster": trial.suggest_categorical('booster', ["gbtree"]),
        "gamma": trial.suggest_float('gamma', 0, 100, step=0.1),
        "tree_method": trial.suggest_categorical('tree_method', ["gpu_hist"]),
        "reg_lambda": trial.suggest_loguniform('reg_lambda', 0.1, 100),
        "reg_alpha": trial.suggest_loguniform('reg_alpha', 0.1, 100),
        "random_state": trial.suggest_categorical('random_state', [42]),
        "n_jobs": trial.suggest_categorical('n_jobs', [4]),
        "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
            }
    
    #opt_params = params
    #opt_params['n_estimators'] = 80000
    
    model = XGBClassifier(**params)

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)],
        #eval_metric='auc',
        verbose=False
    )

    preds = model.predict_proba(np.float32(xvalid))
    yhat = preds[:, 1]
    return roc_auc_score(yvalid, yhat)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_params)

In [ ]:
print("Best value (RMSE): {:.5f}".format(study.best_value))
print("Best params:")

for key, value in study.best_params.items():
    print("{}: {}".format(key, value))

In [ ]:
xgb_params = study.best_params
xgb_params

In [ ]:
from sklearn.metrics import roc_curve, auc

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    xgb = XGBClassifier(**xgb_params, 
          predictor='gpu_predictor', 
          gpu_id=0)

    xgb.fit(
        xtrain, 
        ytrain,
        #eval_metric='auc',
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)], 
        verbose=False
        )
    
    pred_valid = xgb.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = xgb.predict_proba(test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")


In [ ]:
final_preds = np.mean(np.column_stack(preds),axis=1)

In [ ]:
#predictions = xgb.predict_proba(test)

#y_pred = predictions[:,1]

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score


print("Mean absolute error: {}".format(mean_absolute_error(y[:540000], final_preds)))
print("r2 score: {}".format(r2_score(y[:540000], final_preds)))

In [ ]:
#final_output = np.mean(np.column_stack(predictions), axis=1)

In [ ]:
#y_pred

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv') 
#sample.set_index('id', inplace=True)

In [ ]:
submission = sample.copy()

submission['target'] = final_preds

submission

In [ ]:
submission.to_csv('submission.csv', index=False)